In [3]:
import os
import glob
import re

import pandas as pd


MODELS_PATH = './models/java-small-model'
results = []

In [25]:
models = {
        '2020_04_05__0.0001_0.0001_0.001__0026d470': 32,
        '2020_04_05__0.0001_0.0001_0.001__2ecc05a5': 22,
        '2020_04_05__0.0001_0.0001_0.001__e152caf0': 28,
        '2020_04_05__0_0_-1__2167eb3d': 37,
        '2020_04_05__0_0_-1__3cff1009': 19,
        '2020_04_05__0_0_-1__4990422c': 19}

In [12]:
models = {'2020_04_07__0_0_-2.0__subtoken_restricted': 24}

In [17]:
models = ['2020_04_09__0_0_-3.0__subtoken_nodes_restricted']

In [2]:
models = ['2020_04_05__0.0001_0.0001_0.001__e152caf0']

In [53]:
coeff_select = [
                 '2020_04_13__1e-05_0.001_-1__556c7315',
                 '2020_04_13__0.0_0.00032_-1__cb617132',
                 '2020_04_13__1e-05_3e-05_-1__26ecce85',
                 '2020_04_13__1e-05_1e-05_-1__d5f7e692',
                 '2020_04_13__1e-05_0.0001_-1__133b3b4a',
                 '2020_04_13__0.0_0.001_-1__2c6b4a81',
                 '2020_04_13__0.0_3e-05_-1__02bbf89b',
                 '2020_04_13__0.0001_0.00032_-1__46537233',
                 '2020_04_13__3e-05_0.001_-1__46a15b73',
                 '2020_04_13__1e-05_0.00032_-1__88e6c7e3',
                 '2020_04_13__0.0_1e-05_-1__acf02da1',
                 '2020_04_13__3e-05_1e-05_-1__e03e1468',
                 '2020_04_13__0.0_0.001_-1__8d50307b',
                 '2020_04_13__3e-05_0.00032_-1__94edc3d2',
                 '2020_04_13__0.0_3e-05_-1__ad2c6b2a',
                 '2020_04_13__0.0_0.0001_-1__c5a5a4c8',
                 '2020_04_13__0.0001_0.0001_-1__d64fba80',
                 '2020_04_13__0.0001_0.001_-1__ca55734c',
                 '2020_04_13__0.0001_3e-05_-1__f6865928',
                 '2020_04_13__0.0_0.0001_-1__f772040c',
                 '2020_04_13__3e-05_3e-05_-1__7502d002',
                 '2020_04_13__0.0_1e-05_-1__b7e86f3e',
                 '2020_04_13__3e-05_0.0001_-1__f83cfcda',
                 '2020_04_13__0.0_0.00032_-1__8cb14a2d',
                 '2020_04_13__0.0001_1e-05_-1__c7b21e93']

In [98]:
coeff_select = ['2020_04_07__0.1_0_-1__b718b076',
                '2020_04_07__0.01_0_-1__53cfd74e',
                '2020_04_07__0.001_0_-1__0f88a7ef',
                '2020_04_07__0.0001_0_-1__92371c27', 
                '2020_04_07__1e-05_0_-1__81cbb810',
                '2020_04_07__1e-06_0_-1__3866b871',
                '2020_04_07__0_0.1_-1__5810e829',
                '2020_04_07__0_0.01_-1__b303bfa9',
                '2020_04_07__0_0.001_-1__facca85d',
                '2020_04_07__0_0.0001_-1__35a3d3de',
                '2020_04_07__0_1e-05_-1__bfb47368',
                '2020_04_07__0_1e-06_-1__be492856',
                '2020_04_05__0_0_-1__2167eb3d']

In [47]:
vocab_words = ['2020_04_11__0_0_-1__632771e7',
               '2020_04_11__0_0_-1__fa0572aa',
               '2020_04_11__0_0_-1__dc65356a',
               '2020_04_05__0_0_-1__2167eb3d',
               '2020_04_13__0_0_-1__2592fca7',
               '2020_04_13__0_0_-1__01b465bd',
               '2020_04_13__0_0_-1__182434f0']

In [99]:
threshold_select  = ['2020_04_21__1e-05_0.0001_0.001__607f3e5d',
                     '2020_04_21__1e-05_0.0001_0.01__a0ea772d',
                     '2020_04_21__1e-05_0.0001_3e-05__829ecb7e',
                     '2020_04_21__1e-05_0.0001_0.00316__eda7e095',
                     '2020_04_21__1e-05_0.0001_0.0__c845667c',
                     '2020_04_21__1e-05_0.0001_0.0001__68faad49',
                     '2020_04_21__1e-05_0.0001_0.0__4ae1c574',
                     '2020_04_21__1e-05_0.0001_1e-05__2c2e103f',
                     '2020_04_21__1e-05_0.0001_0.00032__2a113711',
                     '2020_04_21__1e-05_0.0001_0.0__5c02e2ea',
                     '2020_04_21__1e-05_0.0001_0.0__6d538951',
                     '2020_04_05__0_0_-1__2167eb3d']


In [100]:
models = threshold_select

# Best epoch and validation

In [101]:
def get_sublist_idx(x, y):
    l1, l2 = len(x), len(y)
    for i in range(l1):
        if x[i:i + l2] == y:
            return i
    return None


non_decimal = re.compile(r'[^\d.]+')

results = []
for model in models:
    out_files = glob.glob(MODELS_PATH + '/' + model + "/*.out")
    if out_files == []:
        print(model)
        raise "No .out files"
    train_out = out_files[0]
    
    with open(train_out) as f:
        text = f.read().split()
        s_i = text.index('---------Sparsification')
        
        Lasso, GroupLasso, Threshold = map(float, [text[s_i + 4], 
                                                   text[s_i + 8],
                                                   text[s_i + 14]])
        
        best_epoch = int(non_decimal.sub('', text[-7]))
        val_prec, val_rec, val_F = map(lambda x: float(non_decimal.sub('', x)), [text[-5], text[-3], text[-1]])
        
        subtoken_words, nodes_words = map(lambda x: int(non_decimal.sub('', x)), [text[6], text[17]])
        
        best_epoch = int(non_decimal.sub('', text[-7]))
        val_prec, val_rec, val_F = map(lambda x: float(non_decimal.sub('', x)), [text[-5], text[-3], text[-1]])
        
        results.append([model, subtoken_words, nodes_words, Lasso, GroupLasso, Threshold, 
                        best_epoch, val_prec, val_rec, val_F])
        
    
    err_files = glob.glob(MODELS_PATH + '/' + model + "/*.err")
    if err_files == []:
        print(model)
        raise "No .out files"
    errors_out = err_files[0]
    
    with open(errors_out) as f:
        text = f.read().split()
        
        index = get_sublist_idx(text, ['Indicators', 'after', str(best_epoch), 'epochs:'])
        
        if not(index):
            raise 'Not found results in .err'
        _, _, nodes_words_left, subtoken_words_left, _, _ = \
                            list(filter(None, re.split(r'[\]\[]', text[index + 17])))
        
        results[-1] += [subtoken_words_left, nodes_words_left]

In [102]:
results = pd.DataFrame(results, 
     columns=['folder', 'SUBTOKEN_words_max', 'NODES_words_max', 
              'Lasso', 'GroupLasso', 'Threshold', 'best_epoch', 'Val_Prec', 'Val_Rec', 'Val_F1',
              'SUBTOKEN_words_left', 'NODES_words_left'])

In [103]:
results

,folder,SUBTOKEN_words_max,NODES_words_max,Lasso,GroupLasso,Threshold,best_epoch,Val_Prec,Val_Rec,Val_F1,SUBTOKEN_words_left,NODES_words_left
0,2020_04_21__1e-05_0.0001_0.001__607f3e5d,73906,323,0.00001,0.0001,1.000000e-03,53,0.48441,0.37956,0.42562,72534,322
1,2020_04_21__1e-05_0.0001_0.01__a0ea772d,73906,323,0.00001,0.0001,1.000000e-02,53,0.48564,0.38030,0.42656,49963,273
2,2020_04_21__1e-05_0.0001_3e-05__829ecb7e,73906,323,0.00001,0.0001,3.162278e-05,59,0.48263,0.38150,0.42614,70635,318
3,2020_04_21__1e-05_0.0001_0.00316__eda7e095,73906,323,0.00001,0.0001,3.162278e-03,41,0.48437,0.38149,0.42682,73867,323
4,2020_04_21__1e-05_0.0001_0.0__c845667c,73906,323,0.00001,0.0001,3.162278e-07,52,0.48208,0.38024,0.42515,73152,322
5,2020_04_21__1e-05_0.0001_0.0001__68faad49,73906,323,0.00001,0.0001,1.000000e-04,49,0.48320,0.38376,0.42778,73588,323
6,2020_04_21__1e-05_0.0001_0.0__4ae1c574,73906,323,0.00001,0.0001,1.000000e-06,44,0.49426,0.39523,0.43924,73865,323
7,2020_04_21__1e-05_0.0001_1e-05__2c2e103f,73906,323,0.00001,0.0001,1.000000e-05,42,0.49592,0.37918,0.42976,73890,323
8,2020_04_21__1e-05_0.0001_0.00032__2a113711,73906,323,0.00001,0.0001,3.162278e-04,34,0.49353,0.37455,0.42589,73906,323
9,2020_04_21__1e-05_0.0001_0.0__5c02e2ea,73906,323,0.00001,0.0001,3.162278e-06,35,0.49572,0.37762,0.42868,73905,323


# Release

In [37]:
for index, row in results.iterrows():
    folder = results.iloc[index]['folder']
    epoch = results.iloc[index]['best_epoch']
    command = f'python code2seq.py --load models/java-small-model/{folder}/model_iter{epoch} --release'
    os.system(f'sbatch --error=./slurm/%j.err --output=./slurm/%j.out -J c2s_release --gres=gpu:1 -c 4 --wrap=\"{command}\"')

# Test

In [88]:
for index, row in results.iterrows():
    folder = results.iloc[index]['folder']
    epoch = results.iloc[index]['best_epoch']
    command = f'python3 code2seq.py --load {MODELS_PATH}/{folder}/model_iter{epoch}.release --test data/java-small/java-small.test.c2s'
    os.system(f'sbatch --error=./slurm/{folder}.err --output=./slurm/{folder}.out -J c2s_test --gres=gpu:1 -c 4 --wrap=\"{command}\"')


# Get test results

In [104]:
import pandas as pd

results['Test_Prec'] = None
results['Test_Rec'] = None
results['Test_F1'] = None

for index, row in results.iterrows():
    folder = results.iloc[index]['folder']
    epoch = results.iloc[index]['best_epoch']
    
    with open(f'./slurm/{folder}.out') as f:
        _, _, _, Lasso, GroupLasso, Threshold, _ = list(filter(None, folder.split('_')))
        text = f.read().split()
        prec_i = text.index('Precision:') + 1
        rec_i = text.index('recall:') + 1   
        f_i = text.index('F1:') + 1
        
        prec, rec, f = list(map(lambda a: float(a.rstrip(',')), [text[prec_i], text[rec_i], text[f_i]]))
        results['Test_Prec'][index] = prec
        results['Test_Rec'][index] = rec
        results['Test_F1'][index] = f

/home/aagusev2/anaconda3/envs/tf_env/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/aagusev2/anaconda3/envs/tf_env/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/aagusev2/anaconda3/envs/tf_env/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [105]:
results[[ 'Lasso', 'GroupLasso', 'Threshold', 'best_epoch', 'Val_Prec', 'Val_Rec', 'Val_F1', 'Test_Prec',
       'Test_Rec', 'Test_F1', 'SUBTOKEN_words_left', 'NODES_words_left']].sort_values(by=['Val_F1'], ascending=False)

,Lasso,GroupLasso,Threshold,best_epoch,Val_Prec,Val_Rec,Val_F1,Test_Prec,Test_Rec,Test_F1,SUBTOKEN_words_left,NODES_words_left
6,0.00001,0.0001,1.000000e-06,44,0.49426,0.39523,0.43924,0.487321,0.382149,0.428374,73865,323
7,0.00001,0.0001,1.000000e-05,42,0.49592,0.37918,0.42976,0.483399,0.377744,0.42409,73890,323
10,0.00001,0.0001,1.000000e-07,31,0.49168,0.38028,0.42886,0.490935,0.376923,0.42644,73906,323
9,0.00001,0.0001,3.162278e-06,35,0.49572,0.37762,0.42868,0.484302,0.376027,0.423351,73905,323
5,0.00001,0.0001,1.000000e-04,49,0.48320,0.38376,0.42778,0.488867,0.382051,0.428909,73588,323
3,0.00001,0.0001,3.162278e-03,41,0.48437,0.38149,0.42682,0.491045,0.377709,0.426984,73867,323
1,0.00001,0.0001,1.000000e-02,53,0.48564,0.38030,0.42656,0.481189,0.377108,0.422838,49963,273
2,0.00001,0.0001,3.162278e-05,59,0.48263,0.38150,0.42614,0.486714,0.387222,0.431305,70635,318
8,0.00001,0.0001,3.162278e-04,34,0.49353,0.37455,0.42589,0.491807,0.374103,0.424956,73906,323
0,0.00001,0.0001,1.000000e-03,53,0.48441,0.37956,0.42562,0.487992,0.384099,0.429857,72534,322


## Get sorted by time folders

In [25]:
import os

models_dir = "./models/java-small-model"
# os.chdir(models_dir)
search_dir = './'
model_folders = filter(os.path.isdir, os.listdir(search_dir))
model_folders = [os.path.join(search_dir, f) for f in model_folders] # add path to each file
model_folders.sort(key=lambda x: os.path.getmtime(x), reverse=True)
back_dir = "./../../"
os.chdir(back_dir)

In [13]:
!pwd

/home/aagusev2/vocabularyForSourceCode/code2seq/models/java-small-model


In [30]:
list(map(lambda s: s[2:], model_folders[:11]))

['2020_04_21__1e-05_0.0001_0.001__607f3e5d',
 '2020_04_21__1e-05_0.0001_0.01__a0ea772d',
 '2020_04_21__1e-05_0.0001_3e-05__829ecb7e',
 '2020_04_21__1e-05_0.0001_0.00316__eda7e095',
 '2020_04_21__1e-05_0.0001_0.0__c845667c',
 '2020_04_21__1e-05_0.0001_0.0001__68faad49',
 '2020_04_21__1e-05_0.0001_0.0__4ae1c574',
 '2020_04_21__1e-05_0.0001_1e-05__2c2e103f',
 '2020_04_21__1e-05_0.0001_0.00032__2a113711',
 '2020_04_21__1e-05_0.0001_0.0__5c02e2ea',
 '2020_04_21__1e-05_0.0001_0.0__6d538951']

In [3]:
def get_logs(job, epoch):
    OUTPUT_FOLDER = './SLURM'
    
    
    
    with open(OUTPUT_FOLDER + f'{job}.out','r') as f:
        line = f.readline()
        while line != 'i'
        
    with open(OUTPUT_FOLDER + f'errors_{job}','r') as f:
        errors_file = f.read()




In [6]:
results

,jobID,lasso,group_lasso,threshold,epochs,Precision,recall,F1,NODES_VOCAB nonzeros,SUBTOKEN_VOCAB nonzeros,Lasso Reg,Group Lasso Reg
0,34508,0.0001,0.0001,1.000000e-03,24,0.47440,0.35912,0.40879,1236,30258,0.225216,0.065769
1,34510,0.0001,0.0001,1.000000e-05,64,0.46884,0.36772,0.41217,1486,74339,0.271612,0.061684
2,34511,0.0001,0.0001,1.000000e-06,75,0.46578,0.37260,0.41401,3986,148216,0.301428,0.062066
3,34512,0.0001,0.0001,1.000000e-07,24,0.46324,0.36765,0.40995,18028,6730523,0.408273,0.073424
4,34513,0.0000,0.0001,1.000000e-05,37,0.46248,0.36882,0.41037,40333,9413252,0.000000,4.115718
5,34514,0.0001,0.0000,1.000000e-05,60,0.47707,0.38251,0.42459,1434,73891,0.316947,0.000000


In [ ]:
python3 code2seq.py --load models/java-large-model/model_iter60.release --test data/java-small/java-small.test.c2s